In [ ]:
import boto3
from botocore.exceptions import ClientError
from IPython.display import display, HTML

client = boto3.client("bedrock-runtime", region_name="eu-west-1")


def print_results_in_html(bedrock_response):
    html_content = "<html><body>"
    html_content += "<h3>Response</h3>"
    html_content += "<pre>"
    for item in bedrock_response:
        if "text" in item:
            html_content += item["text"]
        elif "toolUse" in item:
            html_content += f"Tool Use: {item['toolUse']}"
        elif "toolResult" in item:
            html_content += f"Tool Result: {item['toolResult']}"
        else:
            html_content += str(item)
    html_content += "</pre>"
    html_content += "</body></html>"

    display(HTML(html_content))


## Talk to an LLM
LLMs, or Large Language Models, became famous through ChatGPT. A lot of people feel comfortable using chat to interact with an LLM. The following code block shows how to ask the LLM a question.

In [ ]:
model_id = "eu.amazon.nova-lite-v1:0"

def ask_question(question: str) -> str:
    conversation = [
        {
            "role": "user",
            "content": [{"text": question}],
        },
    ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=conversation,
            inferenceConfig={"maxTokens": 512, "temperature": 0, "topP": 0.9},
        )

        # Extract and print the response text.
        return response["output"]["message"]["content"]

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)


In [ ]:
print_results_in_html(ask_question("When is Jettro available?"))

## Be more specific when formulating your question
Ask a better question by articulating what you want. Tell the LLM your intentions.

In [ ]:
print_results_in_html(ask_question("I need to make an appointment with Jettro. When is he available?"))

## Provide a system prompt with some guidelines
With a system message, you tell the LLM more about the role

In [ ]:
def ask_question_with_system_prompt(question) -> str:
    conversation = [
        {
            "role": "user",
            "content": [{"text": question}],
        },
    ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=conversation,
            system=[
                {
                    "text": (
                        "You are a scheduling assistant. You help in checking the availability of people."
                        "Do not make up availability if you do not know the person's schedule."
                        "Answer in short sentences, stick to the answer to the question."
                    )
                },
            ],
            inferenceConfig={"maxTokens": 512, "temperature": 0, "topP": 0.9},
        )

        # Extract and print the response text.
        return response["output"]["message"]["content"]

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

In [ ]:
print_results_in_html(ask_question_with_system_prompt("When is Jettro available?"))

## Provide a context with the required information
The way you obtain the context is not important for the LLM.

In [ ]:
context = (
    "This is the agenda of people in our office:\n"
    "- Jettro is available on Monday and Thursday;\n"
    "- Joey is available on Tuesday, Thursday, and Friday;\n"
    "- Daniel is available from Monday to Thursday."
)

print_results_in_html(ask_question_with_system_prompt(f"{context}\nWhen is Jettro available?"))

In [ ]:
print_results_in_html(ask_question_with_system_prompt(f"When are Jettro, Joey and Daniel available together?"))

## Give the LLM Memory
By keeping the messages in a memory, the LLM can keep using the context.

In [ ]:
def ask_question_with_memory(question, messages=None) -> (str, list[dict]):
    conversation = messages or []
    conversation.append(
        {
            "role": "user",
            "content": [{"text": question}],
        }
    )

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=conversation,
            system=[
                {
                    "text": (
                        "You are a scheduling assistant. You help in checking the availability of people."
                        "Do not make up availability if you do not know the person's schedule."
                        "Answer in short sentences, stick to the answer to the question."
                    )
                },
            ],
            inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
        )

        # Extract and print the response text.
        response_text = response["output"]["message"]["content"]
        conversation.append(response["output"]["message"])
        return response_text, conversation

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)


conversational_memory = None

In [ ]:
answer, conversational_memory = ask_question_with_memory(question=f"{context}\nWhen is Jettro available?",
                                                         messages=conversational_memory)
print_results_in_html(answer)

In [ ]:
answer, conversational_memory = ask_question_with_memory(question=f"When are Jettro and Joey both available?",
                                                         messages=conversational_memory)
print_results_in_html(answer)

## Give the LLM a Tool to ask schedule information
By using a Tool, we can have a dynamic context. The Tool is a function that the LLM knows how to call.

In [ ]:
# This is the tool
def find_person_availability(name: str) -> str:
    print(f"Finding availability for {name}")
    if name.lower() == "jettro":
        return "Jettro is available on Tuesday and Thursday."
    elif name.lower() == "daniel":
        return "Daniel is available on Monday to Thursday."
    elif name.lower() == "joey":
        return "Joey is available on Thursday and Friday."
    else:
        return "I do not know the availability of this person."


In [ ]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "find_person_availability",
                "description": "Find the availability of a person",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "The name of the person to find availability for.",
                            },
                        },
                        "required": ["name"],
                    }
                }
            }
        }
    ]
}

In [ ]:
def ask_question_with_tool(question, messages=None) -> (str | list[dict], list[dict]):
    conversation = messages or []
    # Add the system message to the conversation
    if question:
        conversation.append(
            {
                "role": "user",
                "content": [{"text": question}],
            }
        )

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=conversation,
            system=[
                {
                    "text": (
                        "You are a scheduling assistant. You help in checking the availability of people."
                        "Do not make up availability if you do not know the person's schedule."
                        "Answer in short sentences, stick to the answer to the question."
                    )
                },
            ],
            inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
            toolConfig=tool_config,
        )
        conversation.append(response["output"]["message"])

        # Check if the response contains a tool call
        stop_reason = response["stopReason"]
        if stop_reason == "tool_use":
            # Extract the tool call information
            tool_calls = []
            for item in response["output"]["message"]["content"]:
                if "toolUse" in item:
                    tool_calls.append(item["toolUse"])
            return tool_calls, conversation
        else:
            # Extract and print the response text.
            response_text = response["output"]["message"]["content"]
            return response_text, conversation

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)


def add_tool_call_to_memory(tool_calls, messages):
    tool_results = []
    for tool_call in tool_calls:
        if tool_call["name"] == "find_person_availability":
            name = tool_call["input"]["name"]
            availability = find_person_availability(name)
            tool_result = {
                "toolResult": {
                    "toolUseId": tool_call["toolUseId"],
                    "content": [{"text": availability}],
                }
            }
            tool_results.append(tool_result)

    messages.append(
        {
            "role": "user",
            "content": tool_results,
        }
    )


In [ ]:
conversational_memory = None

answer, conversational_memory = ask_question_with_tool(question=f"When are Jettro and Joey both available?",
                                                       messages=conversational_memory)
print_results_in_html(answer)


In [ ]:
add_tool_call_to_memory(answer, conversational_memory)

answer, conversational_memory = ask_question_with_tool(question=None,
                                                       messages=conversational_memory)

print_results_in_html(answer)